In [1]:
import os
import datetime
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import pandas as pd
import numpy as np
import random

import dataloader
import utils
from model import generate_model
from optimizer import Adam,SGD
from train_wrapper import train_epoch

In [2]:
config = utils.load_config()

In [3]:
#seed 고정

seed = config['seed']
random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = True    

In [7]:
#Dataset load
X_train = pd.read_csv(config['DATASET']['TRAIN']).drop(labels='label',axis=1)
X_test = pd.read_csv(config['DATASET']['TEST']).drop(labels='label',axis=1)
y_train = pd.read_csv(config['DATASET']['TRAIN'])['label']
y_test = pd.read_csv(config['DATASET']['TEST'])['label']

In [ ]:
batch_size = config['dataloader']['batch_size']
num_workers = config['dataloader']['num_workers']
learning_rate = config['train']['learning_rate']
epochs = config['train']['epochs']

model_name = config['model']['model_name']
model_depth = config['model']['model_depth']
config['save_datetime'] = datetime.datetime.now().strftime("%Y%m%d_%H%M")

In [ ]:
traindata=dataloader.CTDataset(X_train,y_train)
testdata=dataloader.CTDataset(X_test,y_test)

train_dataloader = DataLoader(traindata , batch_size=batch_size, shuffle=True, sampler = None, num_workers=num_workers,pin_memory = True)
test_dataloader = DataLoader(testdata , batch_size=batch_size, shuffle=False, num_workers=num_workers, pin_memory = True)

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

model, _ = generate_model(model_name=model_name,model_depth = model_depth,n_classes=2,resnet_shortcut='B',add_last_fc_num = 0)
model.to(device)

#패러럴 Train 미사용
if torch.cuda.is_available() and config['parallel'] == 1:
    model = nn.DataParallel(model, device_ids = config['gpus'])
    model.to(device)
    
optimizer = Adam(model, learning_rate = learning_rate)
criterion_clf = nn.CrossEntropyLoss().to(device)

In [ ]:
utils.save_messgage(config
                    , model_name=model_name, model_depth = model_depth, n_classes=2, resnet_shortcut='B'
                    , optimizer = 'Adam', lr=learning_rate, criterion_clf='CrossEntropyLoss'
                    , message='비고')

train_epoch(device,train_dataloader,test_dataloader,test_dataloader,model,criterion_clf,optimizer,config,epoch = epochs,num_classes=2)

In [ ]:
#사용한 config 파일 복사
utils.copy_config(config)

In [ ]:
%%javascript
Jupyter.notebook.session.delete();